In [9]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 

<IPython.core.display.Javascript object>

In [10]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
from keras.models import load_model
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os
import glob

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [26]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
lower_case_csv = csv_file.lower()
img_path = "../../images/" + lower_case_csv.split(".")[0]


# Choose column of label for prediction
models = os.listdir("./models/")

mod_menu = {}
for i in range(0, len(models)):
    mod_menu[models[i]] = models[i]

def f(models_menu):
    return models_menu

out2 = interact(f, models_menu=mod_menu)

interactive(children=(Dropdown(description='models_menu', options={'zaslavsk_cnn_Petrographic Fabric_25_64.h5'…

In [25]:
#select model
model = out2.widget.result

#get label
def getLabel(model):
    label_header_array1 = model.split("cnn_")
    label_header_array2 = label_header_array1[1].split("_")
    label_header = label_header_array2[0] 
    return label_header

labelHeader = getLabel(model)                   

In [3]:
# use csv file to grab images/labels
nameCol = df['#img']
predCol = df[label_header]

# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []

# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join(img_path, base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [4]:
# Reshape original input data images for predicting
img_list = np.array(img_list, dtype ="float")/225.0

predictionsMade = []
preds = model.predict(img_list)

# Run all data through the prediction model that was created
for i in range (0,len(img_list)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
    
# Count how many correct predictions were made
#correct = 0
#for i in range (0,len(predictionsMade)):
#    if(predictionsMade[i] == yset[i]):
#        correct += 1 
        
#print("Accuracy: " + str(correct/len(yset)))

In [6]:
preds

array([[7.3146722e-03, 2.2759998e-02, 2.6123662e-02, ..., 2.4285502e-04,
        2.6238686e-04, 4.4138386e-04],
       [8.2373573e-03, 2.3680601e-02, 2.8339103e-02, ..., 2.7124159e-04,
        2.9827471e-04, 4.9952808e-04],
       [5.6405989e-03, 1.8148303e-02, 2.1008486e-02, ..., 1.5238988e-04,
        1.6678484e-04, 2.8657098e-04],
       ...,
       [5.9150276e-03, 1.9931983e-02, 2.2404270e-02, ..., 1.7361315e-04,
        1.8685540e-04, 3.1787110e-04],
       [1.9720931e-02, 4.4948902e-02, 4.9206231e-02, ..., 1.4826967e-03,
        1.5633070e-03, 2.3219495e-03],
       [2.6134825e-03, 1.0256716e-02, 1.2193894e-02, ..., 3.7956270e-05,
        4.2100517e-05, 8.0813763e-05]], dtype=float32)